# Riid! Correct Answers Prediction

We need pandas, numpy, datatable, and a few other libraries.
We also need riiideducation for this particular competititon.

In [ ]:
#!pip install datatable

import riiideducation
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#import datatable as dt
from sklearn import metrics

Get the training data. It's large, so this may take a while.

In [ ]:
%%time

dtypes ={'row_id': 'int64', 
         'timestamp': 'int64', 
         'user_id': 'int32', 
         'content_id': 'int16', 
         'content_type_id': 'int8',
         'task_container_id': 'int16', 
         'user_answer': 'int8', 
         'answered_correctly': 'int8', 
         'prior_question_elapsed_time': 'float32', 
         'prior_question_had_explanation': 'boolean',
        }

#train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv').to_pandas()
data_df = pd.read_csv('../input/riiid-test-answer-prediction/train.csv', nrows=150000, dtype=dtypes)
print("All data:", data_df.shape)

train_df = data_df[:100000]
valid_df = data_df[100000:]
print("Train:", train_df.shape)
print("Valid:", valid_df.shape)

train_df = train_df.drop(train_df[train_df['answered_correctly']==-1].index)

Define some functions to get test data and labels.

In [ ]:
def get_input(df):
    return df[['timestamp', 'content_id', 'content_type_id', 
                       'task_container_id', 'prior_question_elapsed_time', 
                       'prior_question_had_explanation']].fillna(0)

def get_labels(df):
    return df['answered_correctly']

## Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

def train_model():
    model.fit(get_input(train_df), get_labels(train_df))
    train_score = model.score(get_input(train_df), get_labels(train_df))
    valid_score = model.score(get_input(valid_df), get_labels(valid_df))
    print('Train score:', train_score)
    print('Valid score:', valid_score)

Now we train our model:

In [ ]:
model.fit(get_input(train_df), get_labels(train_df))

And we check the accuracy:

In [ ]:
train_score = model.score(get_input(train_df), get_labels(train_df))
valid_score = model.score(get_input(valid_df), get_labels(valid_df))
print('Train score:', train_score)
print('Valid score:', valid_score)

## Test (Validate) Model

We can start an environment and go through iter_test exactly once in the notebook. This is a dummy prediction, to test a submission.

In [ ]:
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df['answered_correctly'] = model.predict(get_input(test_df))
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])